# Team Exsilio


## Task 1

In [ ]:
import linecache
from itertools import islice
import pandas as pd
from pprint import pprint


class CorpusReader:
  """This class indicates the Reader task for Task 1"""

    # Creates the Dictionaries from the Path of the File

  def create_dicts(self, path):
    """
      Returns 2 Dictionaries. Once for the Line and One For the Relation in the File. 
      This is based on how the Data is organized in the supplied Data Files.
    """
    line_d = {}
    rel_d = {}

    with open(path) as f:
      for line in islice(f, 0, None, 4):
        lister = line.split('"')
        line_number = int(lister[0].split('\t')[0])
        line_d[line_number] = ''.join(str(s) for s in lister[1:])
    
    with open(path) as f:
      for i, line in enumerate(islice(f, 1, None, 4)):
        rel_d[i] = line.split('\n')[0]
    
    return (line_d, rel_d)

  def create_dataframe(self, dictionary_to_convert, cols):

    """
      From a Dictionary which is passed, and the desired column to create, this function
      returns a Dataframe.
    """

    dataframe_converted = pd.DataFrame.from_dict(dictionary_to_convert, orient='index', columns = cols)
    dataframe_converted = dataframe_converted.reset_index()
    dataframe_converted = dataframe_converted.drop(columns=['index'])

    return dataframe_converted


  def parse_data(self, path_to_file):

    """
      Invokes the Create Dict and Create Data Frame Function.
      This function is designed to create the Line and Relation Dataframe
    """

    line_dict, rel_dict = self.create_dicts(path_to_file)
    
    line_df = self.create_dataframe(line_dict, ['line'])
    rel_df = self.create_dataframe(rel_dict, ['relation'])

    line_df['relation'] = rel_df['relation']

    return (line_df, rel_df)



In [ ]:
# Task 1

lines = CorpusReader()
ans_df, rel_df = lines.parse_data('/content/test_sentence.txt')

print("The Output of Task 1 is: \n")
print("The Corpus has ", len(ans_df), " sentences\n") 
for index in ans_df.index:
  print("\n----------------------------------------------------------------------------------------------------------------------------------------\n")
  print("The Parsed Line is             : ", ans_df['line'][index])
  print("The Parsed Line has Relation   : ", ans_df['relation'][index])
  print("\n----------------------------------------------------------------------------------------------------------------------------------------\n")

The Output of Task 1 is: 

The Corpus has  2  sentences


----------------------------------------------------------------------------------------------------------------------------------------

The Parsed Line is             :  After handsome renovations at various locales, the <e1>company</e1> has remodeled a church into a <e2>home</e2>.

The Parsed Line has Relation   :  Other

----------------------------------------------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------------------------------------------

The Parsed Line is             :  The <e1>laptop</e1> was inside a protective <e2>plastic bag</e2> made from LDPE number 4, which can be easily recycled.

The Parsed Line has Relation   :  Content-Container(e1,e2)

---------------------------------------------------------------------------------------------------------------------------

# Task 2: All the required Fields

In [ ]:
import nltk
import spacy
from spacy import displacy
from nltk.corpus import wordnet as wn
import re

nltk.download('wordnet')
nltk.download('punkt')


nlp = spacy.load("en_core_web_sm")
sp = spacy.load('en_core_web_sm')

class AllTasks(CorpusReader):

  # Adding a column of tokens to the dataframe
    def create_tokens(self, dataframe):

      """

        For A DataFrame with the column 'line', this function will create tokens
        of the words in that line

        These tokens will be added as a New Column Named 'Tokens' in the DataFrame and will be returned

      """

      tokenize_dict = {}
      filtered_token_dict = {}
      iterator = dataframe.to_dict('dict')['line']
      stopWords = ['e1', '/e1', 'e2', '/e2', '<', '>', '<e1>', '</e1>', '<e2>', '</e2>']
      for key, val in iterator.items():
        tokenize_dict[key] = nltk.word_tokenize(val)

      for key, val in tokenize_dict.items():
        all_tokens = []
        filtered_tokens = []
        for i in range(len(val)):
          if val[i] == '<':
            val[i] = ''.join(val[i:i+3])
        
          all_tokens = [e for e in val if e not in ('e1', 'e2', '/e1', '/e2', '>')]
          filtered_tokens = [word for word in val if word not in stopWords]
          filtered_token_dict[key] = ', '.join(str(word) for word in filtered_tokens)
          tokenize_dict[key] = ', '.join(str(s) for s in all_tokens)

      tokenize_dataframe = self.create_dataframe(tokenize_dict, ['token'])
      filtered_tok_dataframe = self.create_dataframe(filtered_token_dict, ['filtered tokens'])
      
      dataframe['tokens'] = tokenize_dataframe['token']
      dataframe['filtered tokens'] = filtered_tok_dataframe['filtered tokens']

      return dataframe


    def synctactic_parse(self, dataframe, col):
      
      """

        For A DataFrame with the window created, this function will add the Full Synctactic Parse.

        These values will be added as Two Columns Named 'syn_parse' in the DataFrame and will be returned.

      """
      
      for i, val in enumerate(dataframe[col]):
        print("\n----------------------------------------------------------------------------------------------------------------------------------------\n")
        print("Sentence : ", i+1, "\n")

        print("\nText\t\t\t\t : Root Text\t\t\t : Root Dep\t\t\t : Root Head\t\t\t : Root Head Pos\t|\n")
        print("------------------------------------------------------------------------------------------------------------------------------------------\n")
        
        s = sp(''.join(val).replace(',', ''))

        for dep in s.noun_chunks:
          print(dep.text, "\t\t :", dep.root.text, "\t\t\t :", dep.root.dep_, "\t\t\t :", dep.root.head.text, "\t\t\t :", dep.root.head.pos_,"\t |\n")
        print("\n----------------------------------------------------------------------------------------------------------------------------------------\n")
    def create_pos_dep_lemma(self, dataframe, col):
      """

        For A DataFrame with the window created, this function will add the POS and Dep Tags of those words.

        These values will be added as Two Columns Named 'pos' and 'dep' in the DataFrame and will be returned.

      """
      pos_dict = {}
      dep_dict = {}
      lem_dict = {}
      p = []
      d = []
      l = []
      for i, val in enumerate(dataframe[col]):
        s = sp(''.join(val).replace(',', ''))
        for word in s:
          p.append(word.pos_)
          d.append(word.dep_)
          l.append(word.lemma_)
        pos_dict[i] = ', '.join(str(s) for s in p)
        dep_dict[i] = ', '.join(str(s) for s in d)
        lem_dict[i] = ', '.join(str(s) for s in l)
        p = []
        d = []
        l = []
      

      colname1 = col + '_pos' if col in ['e1', 'e2', 'line window'] else 'pos'
      colname2 = col + '_dep' if col in ['e1', 'e2', 'line window'] else 'dep'
      colname3 = col + '_lem' if col in ['e1', 'e2', 'line window'] else 'lem'
      pos_dataframe = self.create_dataframe(pos_dict, [colname1])
      dep_dataframe = self.create_dataframe(dep_dict, [colname2])
      lem_dataframe = self.create_dataframe(lem_dict, [colname3])

      dataframe[colname1] = pos_dataframe[colname1]
      dataframe[colname2] = dep_dataframe[colname2]
      dataframe[colname3] = lem_dataframe[colname3]
      return dataframe


    def create_NER(self, dataframe):

      """

        For A DataFrame with line, this function will extract both the entities.

        These values will be added as Two Columns Named 'e1' and 'e2' in the DataFrame and will be returned.

      """

      dataframe['entities'] = dataframe['line']
      entity_dict = {}
      entity_type = {}

      for i, val in enumerate(dataframe['entities']):
        e1 = re.findall('<e1>(.*?)</e1>', val)
        e2 = re.findall('<e2>(.*?)</e2>', val)
        entity_dict[i+1] = (str(e1[0]), str(e2[0]))
        doc = nlp(e1[0])
        for ent in doc.ents:
          if ent.label_:
            entity_type[i] = (ent.label_, 'NONE')
        
        doc = nlp(e2[0])
        for ent in doc.ents:
          if ent.label_:
            entity_type[i] = (entity_type[i][0], ent.label_)

      entity_dataframe = self.create_dataframe(entity_dict, ['e1', 'e2'])
      entity_type_df   = self.create_dataframe(entity_type, ['e1', 'e2'])

      dataframe = dataframe.drop(columns=['entities'])
      dataframe['e1'] = entity_dataframe['e1']
      dataframe['e2'] = entity_dataframe['e2']
      dataframe['e1_type'] = entity_type_df['e1']
      dataframe['e2_type'] = entity_type_df['e2']

      return dataframe

    def print_all_hyps(self, dataframe, col):

      for i, val in enumerate(ans_df[col]):
        val = val.replace(' ', '')
        string = val.split(',')
        print("\n----------------------------------------------------------------------------------------------------------------------------------------\n")
        print("Sentence : ", i+1, "\n")
        print("\n----------------------------------------------------------------------------------------------------------------------------------------\n")
        for word in string:
          if wn.synsets(word):

            syn = wn.synsets(word)[0]
            print("\n")
            print("Word: ",word)
            print("Holonyms   :", wn.synsets(word)[0].part_holonyms())
            print("Meronyms   :", wn.synsets(word)[0].part_meronyms())
            print("HyperNyms  :", syn.hypernyms())
            print("HypoNyms   :", syn.hyponyms())



    def create_hyper(self, dataframe, col):

      hypernym = {}
      hyper = []
      all_hyper = []

      for i, val in enumerate(ans_df[col]):
        val = val.replace(' ', '')
        string = val.split(',')
        for word in string:
          if wn.synsets(word):

            syn = wn.synsets(word)[0]
            hype = syn.hypernyms()

            if hype:

              for value in hype:
                hyper.append(str(value)[8:-3].split('.')[0])

              all_hyper.append(word + ' : ' + ', '.join(v for v in hyper))
              hyper = []
          
        hypernym[i] = ', '.join(v for v in all_hyper)
        all_hyper = []
      colname = 'hyp'
      hypernym_dataframe = self.create_dataframe(hypernym, [colname])
      dataframe[colname] = hypernym_dataframe[colname]

      return dataframe
# ------------------------------------------------------------------------

    def create_holo(self, dataframe, col):

      holonym = {}
      holo = []
      all_holo = []

      for i, val in enumerate(ans_df[col]):
        val = val.replace(' ', '')
        string = val.split(',')
        for word in string:
          if wn.synsets(word):

            hol = wn.synsets(word)[0].part_holonyms()
            if hol:
              for value in hol:
                holo.append(str(value)[8:-3].split('.')[0])

              all_holo.append(word + ' : ' + ', '.join(v for v in holo))
              hol = []
          
        holonym[i] = ', '.join(v for v in all_holo)
        all_holo = []
      colname = 'holo'
      holonym_dataframe = self.create_dataframe(holonym, [colname])
      dataframe[colname] = holonym_dataframe[colname]

      return dataframe

    def create_mero(self, dataframe, col):

      meronym = {}
      mero = []
      all_mero = []

      for i, val in enumerate(ans_df[col]):
        val = val.replace(' ', '')
        string = val.split(',')
        for word in string:
          if wn.synsets(word):
            mer = wn.synsets(word)[0].part_meronyms()
          
            if mer:
              for value in mer:
                mero.append(str(value)[8:-3].split('.')[0])

              all_mero.append(word + ' : ' + ', '.join(v for v in mero))
              mer = []
        meronym[i] = ', '.join(v for v in all_mero)
        all_mero = []
      colname = 'mero'
      meronym_dataframe = self.create_dataframe(meronym, [colname])
      dataframe[colname] = meronym_dataframe[colname]

      return dataframe


    def create_hypo(self, dataframe, col):

      hyponym = {}
      hypo = []
      all_hypo = []

      for i, val in enumerate(ans_df[col]):
        val = val.replace(' ', '')
        string = val.split(',')
        for word in string:
          if wn.synsets(word):

            syn = wn.synsets(word)[0]
            hyp = syn.hyponyms()

            if hyp:
              for value in hyp:
                hypo.append(str(value)[8:-3].split('.')[0])

              all_hypo.append(word + ' : ' + ', '.join(v for v in hypo))
              hyp = []
        hyponym[i] = ', '.join(v for v in all_hypo)
        all_hypo = []
      colname = 'hypo'
      hyponym_dataframe = self.create_dataframe(hyponym, [colname])
      dataframe[colname] = hyponym_dataframe[colname]

      return dataframe

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
task2 = AllTasks()
ans_df = task2.create_tokens(ans_df)
for index in ans_df.index:
  print("\n----------------------------------------------------------------------------------------------------------------------------------------\n")
  print("The Tokens are          : ", ans_df['tokens'][index])
  print("The Filtered Tokens are : ", ans_df['filtered tokens'][index])
  print("\n----------------------------------------------------------------------------------------------------------------------------------------\n")


----------------------------------------------------------------------------------------------------------------------------------------

The Tokens are          :  After, handsome, renovations, at, various, locales, ,, the, <e1>, company, </e1>, has, remodeled, a, church, into, a, <e2>, home, </e2>, .
The Filtered Tokens are :  After, handsome, renovations, at, various, locales, ,, the, company, has, remodeled, a, church, into, a, home, .

----------------------------------------------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------------------------------------------

The Tokens are          :  The, <e1>, laptop, </e1>, was, inside, a, protective, <e2>, plastic, bag, </e2>, made, from, LDPE, number, 4, ,, which, can, be, easily, recycled, .
The Filtered Tokens are :  The, laptop, was, inside, a, protective, plastic, bag, made, from, LDPE, n

In [ ]:
ans_df = task2.create_pos_dep_lemma(ans_df, 'filtered tokens')
for index in ans_df.index:
  print("\n----------------------------------------------------------------------------------------------------------------------------------------\n")
  print("The Filtered Tokens are     : ", ans_df['filtered tokens'][index], "\n")
  print("The Lemmas are              : ", ans_df['lem'][index], "\n")
  print("The POS Tags are            : ", ans_df['pos'][index], "\n")
  print("The Dependency Parse is     : ", ans_df['dep'][index], "\n")
  print("\n----------------------------------------------------------------------------------------------------------------------------------------\n")


----------------------------------------------------------------------------------------------------------------------------------------

The Filtered Tokens are     :  After, handsome, renovations, at, various, locales, ,, the, company, has, remodeled, a, church, into, a, home, . 

The Lemmas are              :  after, handsome, renovation, at, various, locale,  , the, company, have, remodel, a, church, into, a, home, . 

The POS Tags are            :  ADP, ADJ, NOUN, ADP, ADJ, NOUN, SPACE, DET, NOUN, AUX, VERB, DET, NOUN, ADP, DET, NOUN, PUNCT 

The Dependency Parse is     :  prep, amod, pobj, prep, amod, pobj, , det, nsubj, aux, ROOT, det, dobj, prep, det, pobj, punct 


----------------------------------------------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------------------------------------------

The Filtered Tokens are     :  The, lap

In [ ]:
ans_df

,line,relation,tokens,filtered tokens,pos,dep,lem
0,"After handsome renovations at various locales,...",Other,"After, handsome, renovations, at, various, loc...","After, handsome, renovations, at, various, loc...","ADP, ADJ, NOUN, ADP, ADJ, NOUN, SPACE, DET, NO...","prep, amod, pobj, prep, amod, pobj, , det, nsu...","after, handsome, renovation, at, various, loca..."
1,The <e1>laptop</e1> was inside a protective <e...,"Content-Container(e1,e2)","The, <e1>, laptop, </e1>, was, inside, a, prot...","The, laptop, was, inside, a, protective, plast...","DET, NOUN, AUX, ADP, DET, ADJ, ADJ, NOUN, VERB...","det, nsubj, ROOT, prep, det, amod, amod, pobj,...","the, laptop, be, inside, a, protective, plasti..."


In [ ]:
ans_df = task2.create_NER(ans_df)

for index in ans_df.index:
  print("\n----------------------------------------------------------------------------------------------------------------------------------------\n")
  print("Sentence :", index+1, "\n")
  print("The Entity 1 is              : ", ans_df['e1'][index], "\n")
  print("The Entity 2 is              : ", ans_df['e2'][index], "\n")
  print("The Type of Entity 1 is      : ", ans_df['e1_type'][index], "\n")
  print("The Type of Entity 2 is      : ", ans_df['e2_type'][index], "\n")
  print("\n----------------------------------------------------------------------------------------------------------------------------------------\n")



----------------------------------------------------------------------------------------------------------------------------------------

Sentence : 1 

The Entity 1 is              :  company 

The Entity 2 is              :  home 

The Type of Entity 1 is      :  nan 

The Type of Entity 2 is      :  nan 


----------------------------------------------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------------------------------------------

Sentence : 2 

The Entity 1 is              :  laptop 

The Entity 2 is              :  plastic bag 

The Type of Entity 1 is      :  nan 

The Type of Entity 2 is      :  nan 


----------------------------------------------------------------------------------------------------------------------------------------



In [ ]:
task2.print_all_hyps(ans_df, 'filtered tokens')


----------------------------------------------------------------------------------------------------------------------------------------

Sentence :  1 


----------------------------------------------------------------------------------------------------------------------------------------



Word:  After
Holonyms   : []
Meronyms   : []
HyperNyms  : []
HypoNyms   : []


Word:  handsome
Holonyms   : []
Meronyms   : []
HyperNyms  : []
HypoNyms   : []


Word:  renovations
Holonyms   : []
Meronyms   : []
HyperNyms  : [Synset('improvement.n.02')]
HypoNyms   : [Synset('face_lift.n.02')]


Word:  at
Holonyms   : []
Meronyms   : []
HyperNyms  : [Synset('chemical_element.n.01'), Synset('halogen.n.01')]
HypoNyms   : []


Word:  various
Holonyms   : []
Meronyms   : []
HyperNyms  : []
HypoNyms   : []


Word:  locales
Holonyms   : []
Meronyms   : []
HyperNyms  : [Synset('scene.n.01')]
HypoNyms   : []


Word:  company
Holonyms   : []
Meronyms   : []
HyperNyms  : [Synset('institution.n.01')]
HypoNy

In [ ]:
ans_df = task2.create_hyper(ans_df, 'filtered tokens')

In [ ]:
ans_df = task2.create_mero(ans_df, 'filtered tokens')

In [ ]:
ans_df = task2.create_holo(ans_df, 'filtered tokens')

In [ ]:
ans_df = task2.create_hypo(ans_df, 'filtered tokens')

In [ ]:
ans_df

,line,relation,tokens,filtered tokens,pos,dep,lem,e1,e2,e1_type,e2_type,hyp,mero,holo,hypo
0,"After handsome renovations at various locales,...",Other,"After, handsome, renovations, at, various, loc...","After, handsome, renovations, at, various, loc...","ADP, ADJ, NOUN, ADP, ADJ, NOUN, SPACE, DET, NO...","prep, amod, pobj, prep, amod, pobj, , det, nsu...","after, handsome, renovation, at, various, loca...",company,home,NaN,NaN,"renovations : improvement, at : chemical_eleme...","a : picometer, a : picometer, picometer","a : nanometer, church : nanometer, christendom...","renovations : face_lift, company : face_lift, ..."
1,The <e1>laptop</e1> was inside a protective <e...,"Content-Container(e1,e2)","The, <e1>, laptop, </e1>, was, inside, a, prot...","The, laptop, was, inside, a, protective, plast...","DET, NOUN, AUX, ADP, DET, ADJ, ADJ, NOUN, VERB...","det, nsubj, ROOT, prep, det, amod, amod, pobj,...","the, laptop, be, inside, a, protective, plasti...",laptop,plastic bag,NaN,NaN,"laptop : portable_computer, inside : region, a...","was : picometer, picometer, aberdeen, adams, b...","was : nanometer, christendom, nanometer, unite...","inside : face_lift, broadcasting_company, bure..."


In [ ]:
task2.synctactic_parse(ans_df, 'filtered tokens')


----------------------------------------------------------------------------------------------------------------------------------------

Sentence :  1 


Text				 : Root Text			 : Root Dep			 : Root Head			 : Root Head Pos	|

------------------------------------------------------------------------------------------------------------------------------------------

handsome renovations 		 : renovations 			 : pobj 			 : After 			 : ADP 	 |

various locales 		 : locales 			 : pobj 			 : at 			 : ADP 	 |

the company 		 : company 			 : nsubj 			 : remodeled 			 : VERB 	 |

a church 		 : church 			 : dobj 			 : remodeled 			 : VERB 	 |

a home 		 : home 			 : pobj 			 : into 			 : ADP 	 |


----------------------------------------------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------------------------------------------

Sentence :  2 


Text				 :

# Task 2: Extra Features Extracted

In [ ]:
class Task2Extra(AllTasks):

  def create_window(self, dataframe):

    """

      For A DataFrame with the column 'line', this function will create a Window
      of the words from E1 Tag - 1 to E2 Tag + 1 words.

      This window will be added as a New Column Named 'Line Window' in the DataFrame and will be returned

    """

    window_dict = {}
    for i, val in enumerate(dataframe['line']):
      e1 = re.findall('<e1>(.*?)</e2>', val)
      before = re.findall('\w* ?<e1>', val)
      after = re.findall('</e2> ?\w*', val)
      bef = before[0].replace('<e1>', '')
      aft = after[0].replace('</e2>', '')
      s = e1[0].replace('</e1>', '').replace('<e2>', '')
      window_dict[i] = bef+s+aft


    window_dataframe = self.create_dataframe(window_dict, ['window'])

    dataframe['line window'] = window_dataframe['window']
    return dataframe

  def create_syn(self, dataframe, col):
    """

      For A DataFrame with col, this function will extract synsets of both the entities.

      These values will be added as Two Columns Named 'e1_syn' and 'e2_syn' in the DataFrame and will be returned.

    """

    hypernym_e = {}
    hyper = []
    for i, val in enumerate(dataframe[col]):
      if wn.synsets(val):
        syn = wn.synsets(val)[0]
        s = syn.hypernyms()
        if s:
          for val in s:
            hyper.append(str(val)[8:-3].split('.')[0])

          hypernym_e[i] = ', '.join(v for v in hyper)
          hyper = []
        else:
          hypernym_e[i] = 'None'
      else:
        hypernym_e[i] = 'None'

      
    colname = col + '_syn'

    hypernym_e_dataframe = self.create_dataframe(hypernym_e, [colname])
    dataframe[colname] = hypernym_e_dataframe[colname]

    return dataframe



In [ ]:
task2extra = Task2Extra()

In [ ]:
ans_df['just_relation'] = [x.split('(')[0] for x in ans_df['relation']]
for index in ans_df.index:
  print("\n----------------------------------------------------------------------------------------------------------------------------------------\n")
  print("Sentence :", index+1, "\n")
  print("The Relation with Direction is      : ", ans_df['relation'][index], "\n")
  print("The Relation without Direction is   : ", ans_df['just_relation'][index], "\n")
  print("\n----------------------------------------------------------------------------------------------------------------------------------------\n")


----------------------------------------------------------------------------------------------------------------------------------------

Sentence : 1 

The Relation with Direction is      :  Other 

The Relation without Direction is   :  Other 


----------------------------------------------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------------------------------------------

Sentence : 2 

The Relation with Direction is      :  Content-Container(e1,e2) 

The Relation without Direction is   :  Content-Container 


----------------------------------------------------------------------------------------------------------------------------------------



In [ ]:
ans_df = task2extra.create_window(ans_df)
for index in ans_df.index:
  print("\n----------------------------------------------------------------------------------------------------------------------------------------\n")
  print("Sentence :", index+1, "\n")
  print("The Full Line is           : ", ans_df['line'][index], "\n")
  print("The Created Window is      : ", ans_df['line window'][index], "\n")
  print("\n----------------------------------------------------------------------------------------------------------------------------------------\n")


----------------------------------------------------------------------------------------------------------------------------------------

Sentence : 1 

The Full Line is           :  After handsome renovations at various locales, the <e1>company</e1> has remodeled a church into a <e2>home</e2>.
 

The Created Window is      :  the company has remodeled a church into a home 


----------------------------------------------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------------------------------------------

Sentence : 2 

The Full Line is           :  The <e1>laptop</e1> was inside a protective <e2>plastic bag</e2> made from LDPE number 4, which can be easily recycled.
 

The Created Window is      :  The laptop was inside a protective plastic bag made 


------------------------------------------------------------------------------------------

In [ ]:
ans_df = task2extra.create_syn(ans_df, 'e1')
ans_df = task2extra.create_syn(ans_df, 'e2')

for index in ans_df.index:
  print("\n----------------------------------------------------------------------------------------------------------------------------------------\n")
  print("Sentence :", index+1, "\n")
  print("The Entity 1 is           : ", ans_df['e1'][index], "\n")
  print("The Synset for E1 are     : ", ans_df['e1_syn'][index], "\n")
  print("The Entity 2 is           : ", ans_df['e2'][index], "\n")
  print("The Synset for E2 are     : ", ans_df['e2_syn'][index], "\n")
  print("\n----------------------------------------------------------------------------------------------------------------------------------------\n")


----------------------------------------------------------------------------------------------------------------------------------------

Sentence : 1 

The Entity 1 is           :  company 

The Synset for E1 are     :  institution 

The Entity 2 is           :  home 

The Synset for E2 are     :  residence 


----------------------------------------------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------------------------------------------

Sentence : 2 

The Entity 1 is           :  laptop 

The Synset for E1 are     :  portable_computer 

The Entity 2 is           :  plastic bag 

The Synset for E2 are     :  None 


----------------------------------------------------------------------------------------------------------------------------------------



In [ ]:
ans_df = task2extra.create_pos_dep_lemma(ans_df, 'line window')


for index in ans_df.index:
  print("\n----------------------------------------------------------------------------------------------------------------------------------------\n")
  print("Sentence :", index+1, "\n")
  print("The Line is                      : ", ans_df['line'][index], "\n")
  print("The Line Window                  : ", ans_df['line window'][index], "\n")
  print("The Shortest Depedency Path is   : ", ans_df['line window_dep'][index], "\n")
  print("The POS Tags for the Window is   : ", ans_df['line window_pos'][index], "\n")
  print("\n----------------------------------------------------------------------------------------------------------------------------------------\n")


----------------------------------------------------------------------------------------------------------------------------------------

Sentence : 1 

The Line is                      :  After handsome renovations at various locales, the <e1>company</e1> has remodeled a church into a <e2>home</e2>.
 

The Line Window                  :  the company has remodeled a church into a home 

The Shortest Depedency Path is   :  det, nsubj, aux, ROOT, det, dobj, prep, det, pobj 

The POS Tags for the Window is   :  DET, NOUN, AUX, VERB, DET, NOUN, ADP, DET, NOUN 


----------------------------------------------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------------------------------------------

Sentence : 2 

The Line is                      :  The <e1>laptop</e1> was inside a protective <e2>plastic bag</e2> made from LDPE number 4, which can be eas